
# Update maintainers

Below are some ad-hoc code chunks to assist me in mass-updating the maintainers of the conda-forge R recipes

In [1]:
import os
import subprocess as sp
import sys
import time

In [2]:
# Authenticate with GitHub

import github

gh_token = os.environ['GH_TOKEN']
gh = github.Github(gh_token)
gh_me = gh.get_user()
print("Authenticated as %s"%(gh_me.login))

Authenticated as jdblischak


In [3]:
# Check rate limit
gh.get_rate_limit()

RateLimit(core=Rate(reset=2022-01-03 03:48:58, remaining=4201, limit=5000))

In [4]:
# Search
# https://pygithub.readthedocs.io/en/latest/github.html?highlight=search#github.MainClass.Github.search_code
results = gh.search_code(query="org:conda-forge language:YAML jdblischak conda-forge/r")
results.totalCount

30

In [5]:
# Repositories to keep
skip = ['r-feedstock',
        'r-base-feedstock',
        'r-git2r-feedstock',
        'r-knitr-feedstock',
        'r-rmarkdown-feedstock',
        'r-workflowr-feedstock',
        'r-reshape-feedstock',
        'r-lokern-feedstock',
        'r-densityclust-feedstock',
        'r-rtensor-feedstock']

In [6]:
# Convert search results to dict of repository objects
dict_repo = {}
limit = 55
counter = 0
for r in results:
    repo = r.repository
    name_repo = os.path.basename(repo.full_name)
    if name_repo in skip:
        continue
    if name_repo[:2] != "r-":
        continue
    dict_repo[name_repo] = repo
    counter += 1
    if counter == limit:
        break
    time.sleep(0.1)
print(len(dict_repo))

24


In [7]:
# Fork repositories
for repo in dict_repo.values():
    print(repo.full_name)
    gh_me.create_fork(repo)
    time.sleep(0.5)

conda-forge/r-klar-feedstock
conda-forge/r-pmcmr-feedstock
conda-forge/r-fingerprint-feedstock
conda-forge/r-timedate-feedstock
conda-forge/r-uniqueatommat-feedstock
conda-forge/r-oriclust-feedstock
conda-forge/r-survivalroc-feedstock
conda-forge/r-runit-feedstock
conda-forge/r-anomalydetection-feedstock
conda-forge/r-cooccur-feedstock
conda-forge/r-bfa-feedstock
conda-forge/r-downloader-feedstock
conda-forge/r-pathological-feedstock
conda-forge/r-maldiquant-feedstock
conda-forge/r-hunspell-feedstock
conda-forge/r-xnomial-feedstock
conda-forge/r-conicfit-feedstock
conda-forge/r-chemometricswithr-feedstock
conda-forge/r-objectproperties-feedstock
conda-forge/r-d3heatmap-feedstock
conda-forge/r-ppls-feedstock
conda-forge/r-rpostgresql-feedstock
conda-forge/r-ggsignif-feedstock
conda-forge/r-tensr-feedstock


In [8]:
# Clone repositories
for feedstock in dict_repo.keys():
    print(feedstock)
    localdir="/tmp/%s"%(feedstock)
    remote = "git@github.com:jdblischak/%s.git"%(feedstock)
    cmd = ["git", "clone", "--quiet", remote, localdir]
    if not os.path.exists(localdir):
        sp.run(cmd)

r-klar-feedstock
r-pmcmr-feedstock
r-fingerprint-feedstock
r-timedate-feedstock
r-uniqueatommat-feedstock
r-oriclust-feedstock
r-survivalroc-feedstock
r-runit-feedstock
r-anomalydetection-feedstock
r-cooccur-feedstock
r-bfa-feedstock
r-downloader-feedstock
r-pathological-feedstock
r-maldiquant-feedstock
r-hunspell-feedstock
r-xnomial-feedstock
r-conicfit-feedstock
r-chemometricswithr-feedstock
r-objectproperties-feedstock
r-d3heatmap-feedstock
r-ppls-feedstock
r-rpostgresql-feedstock
r-ggsignif-feedstock
r-tensr-feedstock


In [9]:
conda_forge_r = {
    "johanneskoester",
    "bgruening",
    "daler",
    "jdblischak",
    "cbrueffer",
    "dbast",
    "dpryan79"
}

In [10]:
# Update meta.yaml
for feedstock in dict_repo.keys():
    print(feedstock)
    file_meta = "/tmp/" + feedstock + "/recipe/meta.yaml"

    # read
    handle_meta = open(file_meta, "r")
    lines_meta = handle_meta.readlines()
    handle_meta.close()

    # write back non-filtered lines
    handle_meta = open(file_meta, "w")
    for line in lines_meta:
        keep = True
        for maintainer in conda_forge_r:
            if "- %s"%(maintainer) in line.lower(): 
                keep = False
        if keep:
            handle_meta.write(line)

    handle_meta.close()
    

r-klar-feedstock
r-pmcmr-feedstock
r-fingerprint-feedstock
r-timedate-feedstock
r-uniqueatommat-feedstock
r-oriclust-feedstock
r-survivalroc-feedstock
r-runit-feedstock
r-anomalydetection-feedstock
r-cooccur-feedstock
r-bfa-feedstock
r-downloader-feedstock
r-pathological-feedstock
r-maldiquant-feedstock
r-hunspell-feedstock
r-xnomial-feedstock
r-conicfit-feedstock
r-chemometricswithr-feedstock
r-objectproperties-feedstock
r-d3heatmap-feedstock
r-ppls-feedstock
r-rpostgresql-feedstock
r-ggsignif-feedstock
r-tensr-feedstock


In [11]:
# Update .github/CODEOWNERS
for feedstock in dict_repo.keys():
    print(feedstock)
    file_owners = "/tmp/" + feedstock + "/.github/CODEOWNERS"

    if not os.path.exists(file_owners):
        continue

    # read (it's a single line)
    handle_owners = open(file_owners, "r")
    line_owners = handle_owners.readline()
    handle_owners.close()

    for maintainer in conda_forge_r:
        line_owners = line_owners.replace(" @%s"%(maintainer), "")
        line_owners = line_owners.replace(" @%s"%(maintainer.capitalize()), "")

    # write
    handle_owners = open(file_owners, "w")
    handle_owners.write(line_owners)
    handle_owners.close()

r-klar-feedstock
r-pmcmr-feedstock
r-fingerprint-feedstock
r-timedate-feedstock
r-uniqueatommat-feedstock
r-oriclust-feedstock
r-survivalroc-feedstock
r-runit-feedstock
r-anomalydetection-feedstock
r-cooccur-feedstock
r-bfa-feedstock
r-downloader-feedstock
r-pathological-feedstock
r-maldiquant-feedstock
r-hunspell-feedstock
r-xnomial-feedstock
r-conicfit-feedstock
r-chemometricswithr-feedstock
r-objectproperties-feedstock
r-d3heatmap-feedstock
r-ppls-feedstock
r-rpostgresql-feedstock
r-ggsignif-feedstock
r-tensr-feedstock


In [12]:
# Update README.md
for feedstock in dict_repo.keys():
    print(feedstock)
    file_readme = "/tmp/" + feedstock + "/README.md"

    # read
    handle_readme = open(file_readme, "r")
    lines_readme = handle_readme.readlines()
    handle_readme.close()

    # write back non-filtered lines
    handle_readme = open(file_readme, "w")
    for line in lines_readme:
        keep = True
        for maintainer in conda_forge_r:
            if "* [@%s](https://github.com/%s/)"%(maintainer, maintainer) in line.lower(): 
                keep = False
        if keep:
            handle_readme.write(line)

    handle_readme.close()

r-klar-feedstock
r-pmcmr-feedstock
r-fingerprint-feedstock
r-timedate-feedstock
r-uniqueatommat-feedstock
r-oriclust-feedstock
r-survivalroc-feedstock
r-runit-feedstock
r-anomalydetection-feedstock
r-cooccur-feedstock
r-bfa-feedstock
r-downloader-feedstock
r-pathological-feedstock
r-maldiquant-feedstock
r-hunspell-feedstock
r-xnomial-feedstock
r-conicfit-feedstock
r-chemometricswithr-feedstock
r-objectproperties-feedstock
r-d3heatmap-feedstock
r-ppls-feedstock
r-rpostgresql-feedstock
r-ggsignif-feedstock
r-tensr-feedstock


In [13]:
%%bash

# Commit updated recipes to branch "update-maintainers"

for repo in /tmp/r-*-feedstock
do
  echo "$repo"
  cd "$repo"
  git checkout --quiet -b "update-maintainers"
  git add .github/CODEOWNERS README.md recipe/meta.yaml
  git commit --quiet -m "[ci skip] [skip ci] ***NO_CI*** Update maintainers"
done

/tmp/r-anomalydetection-feedstock
/tmp/r-bfa-feedstock
/tmp/r-chemometricswithr-feedstock
/tmp/r-conicfit-feedstock
/tmp/r-cooccur-feedstock
/tmp/r-d3heatmap-feedstock
/tmp/r-downloader-feedstock
/tmp/r-fingerprint-feedstock
/tmp/r-ggsignif-feedstock
/tmp/r-hunspell-feedstock
/tmp/r-klar-feedstock
/tmp/r-maldiquant-feedstock
/tmp/r-objectproperties-feedstock
/tmp/r-oriclust-feedstock
/tmp/r-pathological-feedstock
/tmp/r-pmcmr-feedstock
/tmp/r-ppls-feedstock
/tmp/r-rpostgresql-feedstock
/tmp/r-runit-feedstock
/tmp/r-survivalroc-feedstock
/tmp/r-tensr-feedstock
/tmp/r-timedate-feedstock
/tmp/r-uniqueatommat-feedstock
/tmp/r-xnomial-feedstock


In [14]:
%%bash

# Push to GitHub

for repo in /tmp/r-*-feedstock
do
  echo "$repo"
  cd "$repo"
  git push --quiet origin update-maintainers
done

/tmp/r-anomalydetection-feedstock
/tmp/r-bfa-feedstock
/tmp/r-chemometricswithr-feedstock
/tmp/r-conicfit-feedstock
/tmp/r-cooccur-feedstock
/tmp/r-d3heatmap-feedstock
/tmp/r-downloader-feedstock
/tmp/r-fingerprint-feedstock
/tmp/r-ggsignif-feedstock
/tmp/r-hunspell-feedstock
/tmp/r-klar-feedstock
/tmp/r-maldiquant-feedstock
/tmp/r-objectproperties-feedstock
/tmp/r-oriclust-feedstock
/tmp/r-pathological-feedstock
/tmp/r-pmcmr-feedstock
/tmp/r-ppls-feedstock
/tmp/r-rpostgresql-feedstock
/tmp/r-runit-feedstock
/tmp/r-survivalroc-feedstock
/tmp/r-tensr-feedstock
/tmp/r-timedate-feedstock
/tmp/r-uniqueatommat-feedstock
/tmp/r-xnomial-feedstock


remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-anomalydetection-feedstock/pull/new/update-maintainers        
remote: 
remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-bfa-feedstock/pull/new/update-maintainers        
remote: 
remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-chemometricswithr-feedstock/pull/new/update-maintainers        
remote: 
remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-conicfit-feedstock/pull/new/update-maintainers        
remote: 
remote: 
remote: Create a pull request for 'update-maintainers' on GitHub by visiting:        
remote:      https://github.com/jdblischak/r-cooccur-feedstock/pull/new/updat

In [32]:
# Open pull requests
# https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html?highlight=create_pull#github.Repository.Repository.create_pull
# https://docs.github.com/en/rest/reference/pulls#create-a-pull-request
# https://pygithub.readthedocs.io/en/latest/examples/PullRequest.html?highlight=create%20a%20pull%20request#create-a-new-pull-request

body = """
Remove individually listed maintainers that are members of the conda-forge/r team

Followed a similar strategy from the docs for updating the maintainers. Unfortunately it doesn't appear possible to skip the CI jobs in the PR itself, even the example PR had jobs run

https://conda-forge.org/docs/maintainer/updating_pkgs.html#updating-the-maintainer-list
"""

feedstocks = list(dict_repo.keys())

for feedstock in feedstocks[-4:]:
    repo = dict_repo[feedstock]
    print(repo.full_name)

    pr = repo.create_pull(
        title="[ci skip] [skip ci] ***NO_CI*** Update maintainers",
        body=body,
        base="master",
        head="jdblischak:update-maintainers"
    )

    time.sleep(10)

    # Merge pull request
    # https://pygithub.readthedocs.io/en/latest/github_objects/PullRequest.html#github.PullRequest.PullRequest.merge
    # https://docs.github.com/en/rest/reference/pulls#merge-a-pull-request
    pr.merge(
      sha=pr.head.sha,
      merge_method="squash"
    )

    time.sleep(10)

conda-forge/r-ppls-feedstock
conda-forge/r-rpostgresql-feedstock
conda-forge/r-ggsignif-feedstock
conda-forge/r-tensr-feedstock
